In [4]:
import os
import cv2

def crop_and_save(image_path, txt_path, output_dir, crop_size=128):
    # Read image (supports .jpg, .png, etc.)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return
    
    img_height, img_width = image.shape[:2]
    
    # Read annotations (YOLO format: [class, x_center, y_center, width, height])
    annotations = []
    if os.path.exists(txt_path):
        with open(txt_path, 'r') as f:
            annotations = [line.strip().split() for line in f.readlines()]
    
    # Create output subfolders
    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)
    
    # Split into 128x128 crops
    crop_num = 0
    for y in range(0, img_height, crop_size):
        for x in range(0, img_width, crop_size):
            crop = image[y:y+crop_size, x:x+crop_size]
            if crop.shape[0] != crop_size or crop.shape[1] != crop_size:
                continue  # Skip incomplete edge crops
            
            # Save cropped image (as .jpg)
            base_name = os.path.splitext(os.path.basename(image_path))[0]
            crop_name = f"{base_name}_{crop_num}.jpg"  # Now saves as JPG
            cv2.imwrite(os.path.join(output_dir, "images", crop_name), crop, [cv2.IMWRITE_JPEG_QUALITY, 95])
            
            # Process annotations for this crop
            crop_annotations = []
            for ann in annotations:
                class_id, x_center, y_center, w, h = ann
                x_center, y_center, w, h = float(x_center), float(y_center), float(w), float(h)
                
                # Convert to absolute coordinates (512 scale)
                abs_x_center = x_center * img_width
                abs_y_center = y_center * img_height
                abs_w = w * img_width
                abs_h = h * img_height
                
                # Check if bbox is fully inside the current crop
                x_min = abs_x_center - abs_w/2
                y_min = abs_y_center - abs_h/2
                x_max = abs_x_center + abs_w/2
                y_max = abs_y_center + abs_h/2
                
                if x_min >= x and x_max <= x + crop_size and y_min >= y and y_max <= y + crop_size:
                    # Adjust coordinates to 128x128 crop
                    new_x_center = (abs_x_center - x) / crop_size
                    new_y_center = (abs_y_center - y) / crop_size
                    new_w = abs_w / crop_size
                    new_h = abs_h / crop_size
                    
                    crop_annotations.append(f"{class_id} {new_x_center:.6f} {new_y_center:.6f} {new_w:.6f} {new_h:.6f}")
            
            # Save annotations for this crop
            with open(os.path.join(output_dir, "labels", f"{crop_name.replace('.jpg', '.txt')}"), 'w') as f:
                f.write("\n".join(crop_annotations))
            
            crop_num += 1

def process_folder(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg"):  # Focus on JPGs only
            image_path = os.path.join(input_dir, filename)
            txt_path = os.path.join(input_dir, filename.replace(".jpg", ".txt"))
            crop_and_save(image_path, txt_path, output_dir)

# Example usage
input_folder = "C:/Users/LENOVO/Downloads/Programs/negative"  # Folder with .jpg and .txt files
output_folder = "C:/Users/LENOVO/Downloads/Programs/negative"  # Output directory
process_folder(input_folder, output_folder)
print(f"Done! Cropped JPGs and annotations saved to {output_folder}")

Done! Cropped JPGs and annotations saved to C:/Users/LENOVO/Downloads/Programs/negative
